# Convolutional Neural Networks
   Taught By Vignesh Hari , Using MNIST Dataset
   
## Requirements
We'll be using 
Tensorflow 1.9.0


In [17]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

## MNIST
Downloading and Using the MNIST Dataset 
The MNIST Dataset includes images of handwritten digits, they are 28 x 28 in size and have only one channel ( ie Black and White )  
One Hot Encoding Means that the y value is stored as a position of a list rather than a value   
eg : [0,0,1] instead of 2  ||  [0,1,0] instead of 1 || and so on ..  

In [18]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True )

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## Input Data
The data to be used in the Tensorflow Session is taken as a Tensorflow Placeholder , Since the images are 28 x 28 in size , We use (28 * 28) sized vectors to store the unrolled data

In [28]:
x_inp = tf.placeholder(tf.float32, [None, 784]) # 28 * 28 Input nodes
y = tf.placeholder(tf.float32, [None, 10]) # 10 input classes

## Reshaping
To use a Convolutional Neural Net we have to Convolute + Pool + Fully Connect ( Will be Explained later )

Convoluting means creating feature maps from the inputs by creating small frames and running it through the data, The Data has to be converted into its original dimention for this , hence we reshape the data back to 28 x 28

In [20]:
x = tf.reshape(x_inp, [-1, 28, 28, 1])

## Weights


In [21]:
weight_conv1  = tf.Variable(tf.random_normal([5,5,1,32])) 
weight_conv2  = tf.Variable(tf.random_normal([5,5,32,64]))
weight_fc     = tf.Variable(tf.random_normal([7*7*64,784]))
weight_out    = tf.Variable(tf.random_normal([784,10]))

## Biases


In [22]:
bias_conv1  = tf.Variable(tf.random_normal([32])) 
bias_conv2  = tf.Variable(tf.random_normal([64]))
bias_fc     = tf.Variable(tf.random_normal([784]))
bias_out    = tf.Variable(tf.random_normal([10]))

## Layer 1


In [23]:
conv1     = tf.add(tf.nn.conv2d(x , weight_conv1 , strides=[1,1,1,1] , padding="SAME") , bias_conv1) 
max_pool1 = tf.nn.max_pool(conv1 , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding="SAME") 

## Layer 2

In [24]:
conv2     = tf.add(tf.nn.conv2d(max_pool1 , weight_conv2 , strides=[1,1,1,1] , padding="SAME") , bias_conv2)
max_pool2 = tf.nn.max_pool(conv2 , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding="SAME") 

In [25]:
fc = tf.reshape(max_pool2 , [-1 , 7*7*64 ])
fc = tf.nn.relu( tf.add(tf.matmul(fc , weight_fc) , bias_fc) )

In [26]:
out = tf.matmul(fc , weight_out) + bias_out

In [27]:
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=out, labels=y) )
optimizer = tf.train.AdamOptimizer().minimize(cost)


In [13]:
correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

In [14]:
'''
Initialise tensoflow session
'''
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:


hm_epochs = 10
batch_size = 128

for epoch in range(hm_epochs):
    epoch_loss = 0
    for _ in range(int(mnist.train.num_examples/batch_size)):
        epoch_x, epoch_y = mnist.train.next_batch(batch_size)
        _, c = sess.run([optimizer, cost], feed_dict={x_inp: epoch_x, y: epoch_y})
        epoch_loss += c

    print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

('Epoch', 0, 'completed out of', 1, 'loss:', 1781413.0296020508)


In [16]:
print('Accuracy:',sess.run(accuracy , feed_dict={x_inp:mnist.test.images, y:mnist.test.labels}))


('Accuracy:', 0.9274)
